In [3]:
import requests 
import pandas as pd

In [24]:
BASE_API_URL="https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx" 

countries = [    'Afghanistan',    'Bangladesh',    'Bhutan',    'Brunei Darussalam',    'Cambodia',    'Federated States of Micronesia',    'Fiji',    'India',    'Indonesia',    'Kiribati',    'Lao People\'s Democratic Republic',    'Malaysia',    'Myanmar',    'Nepal',    'Pakistan',    'Papua New Guinea',    'Philippines',    'Solomon Islands',    'Sri Lanka',    'Timor-Leste',    'Tonga',    'Uzbekistan',    'Vanuatu',    'Viet Nam',    'Yemen']

data = []    

def is_country_exists(country_name):
    API= "https://osmose.openstreetmap.fr/api/0.3/countries"
    response = requests.get(API).json()
    if country_name.lower() not in response["countries"]:
        print( f"{country_name} doesn't exists on osmose" )
        return False
    return True

def fetch_country(country_name):
    country_name=country_name.lower().replace(" ", "_") # lower case everyname and replace spaces 
    if is_country_exists(country_name):
        call_api_url= BASE_API_URL + f"&country={country_name}"
        print(f"Fetching {call_api_url}")
        response = requests.get(call_api_url)
        data.extend(response.json()["errors_groups"])
        return True

#loop through each country specified 
for country in countries :
    # fetch country data
    fetch_country(country)


# Convert the collected response to a pandas DataFrame
df = pd.DataFrame(data)
print(df. columns. values)

df = df.groupby(['menu', 'country'], as_index=False)['count'].sum()

pivot = df.pivot(index='menu', columns='country', values='count')


Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=afghanistan
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=bangladesh
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=bhutan
brunei_darussalam doesn't exists on osmose
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=cambodia
federated_states_of_micronesia doesn't exists on osmose
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=fiji
india doesn't exists on osmose
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=indonesia
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=kiribati
lao_people's_democratic_republic doesn't exists on osmose
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=malaysia
Fetching https://osmose.openstreetmap.fr/en/issues/open.json?item=xxxx&country=myanmar
Fetching https://osmo

In [25]:
print(df.head(5))
print(pivot.head(5))
pivot.to_csv('final.csv',index=True)

                menu      country  count
0  1-member relation  afghanistan      5
1  1-member relation   bangladesh     60
2  1-member relation       bhutan      5
3  1-member relation     cambodia     16
4  1-member relation         fiji      5
country            afghanistan  bangladesh  bhutan  cambodia   fiji  kiribati  \
menu                                                                            
1-member relation          5.0        60.0     5.0      16.0    5.0      11.0   
URL                        8.0       178.0    47.0      99.0    6.0       1.0   
access                     6.0       121.0    24.0      30.0    2.0       7.0   
addresses                  7.0      1354.0    44.0      40.0  144.0       1.0   
almost junction          229.0      2065.0    23.0     201.0    8.0       1.0   

country            malaysia  myanmar   nepal  pakistan  papua_new_guinea  \
menu                                                                       
1-member relation     149.0     36